## 자연어 추론

문장 쌍 분류로 2가지 문장이 주어지면 

해당 문장 사이의 관계가 어떤 범주인지 분류

ex) 

- 나 공원에 갔어 + 지금 침대 위야 > 거짓
- 나 일하고 있어 + 나는 개발자야 > 중립
- 나 공원에 갔어 + 공원에서 친구를 만났어 > 참

In [10]:
import torch
from pytorch_lightning import Trainer, LightningModule
from pytorch_lightning.callbacks import ModelCheckpoint
from transformers import PreTrainedModel, BertTokenizer, BertConfig, BertForSequenceClassification, set_seed
from transformers.optimization import AdamW
from Korpora import Korpora
import os, csv, re
from dataclasses import dataclass
from typing import List, Optional
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.optim.lr_scheduler import ExponentialLR
import logging

In [4]:
current_dir = !pwd
current_dir = current_dir[0]
current_dir

'/tf/notebooks/NLP'

In [7]:
@dataclass
class Args:
    pretrained_model_name = 'beomi/kcbert-base'
    downstream_task_name = 'pair-classification'
    downstream_corpus_name = 'klue-nli'
    downstream_model_dir = f'{current_dir}/checkpoint-paircls'
    batch_size = 32
    learning_rate = 5e-5
    max_seq_length = 64
    epochs = 5
    seed = 7
args = Args()
args.seed

7

In [9]:
set_seed(args.seed)

In [11]:
logger = logging.getLogger('nli')
logger.setLevel(logging.INFO)
logger.info("Training/evaluation parameters %s", args)

In [13]:
download_info = {
        "train": {
            "googledrive_file_id": "18LhrHaPEW0VITMPfnwKXJ6bNuklBdi4U",
            "fname": "klue_nli_train.json",
        },
        "val": {
            "googledrive_file_id": "1UKIDAFOFuDSah7A66FZXSA8XUWUHhBAd",
            "fname": "klue_nli_dev.json",
        }
    }